# Finfiz

## Traemos los datos de la WEB

In [1]:
import requests
import pandas as pd
import tqdm, sys, os
import numpy as np

In [2]:
def scrapear(pags=range(1,3)):
    agents = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) '
    agents += 'Chrome/50.0.2661.75 Safari/537.36'
    header = {"User-Agent": agents, "X-Requested-With": "XMLHttpRequest"}
    vista_simple = '111'
    vista_full = '152&c='+','.join([str(s) for s in list(range(0,71))])

    df = pd.DataFrame()

    paginas = 1000
    with tqdm.tqdm(total=len(pags), file=sys.stdout) as pbar:
        for pagina in pags:
            pbar.update()

            r = str((pagina)*20 + 1)
            url = f'https://finviz.com/screener.ashx?v={vista_full}&r={r}'
            r = requests.get(url, headers=header)
            tablas = pd.read_html(r.text)

            for tabla in tablas:
                valor = tabla[0].values[0]
                if valor == 'No.':
                    data = tabla

                try:
                    if valor.find('Total: ') == 0:
                        paginas = int(valor[7:valor.find('#')-1])//20 +1
                except:
                    pass

            if pagina == paginas:
                break

            data.columns=data.loc[0]
            data = data.drop(0).drop('No.', axis=1).set_index('Ticker')
            df = pd.concat([df,data])

    return df

In [3]:
finviz = scrapear(pags=range(390))

 98%|██████████████████████████████████████████████████████████████████████████████▎ | 382/390 [03:57<00:04,  1.61it/s]


## Persistimos tabla Original

In [4]:
from sqlalchemy import create_engine

sql_engine = create_engine('mysql+pymysql://root:@localhost/curso_2_clase_8')
sql_conn = sql_engine.connect()

In [5]:
finviz.to_sql(con=sql_conn, name='finviz', if_exists='replace')

In [6]:
%%time
finviz = pd.read_sql('finviz', sql_conn)

Wall time: 1.82 s


In [7]:
finviz

,Ticker,Company,Sector,Industry,Country,Market Cap,P/E,Fwd P/E,PEG,P/S,...,Gap,Recom,Avg Volume,Rel Volume,Price,Change,Volume,Earnings,Target Price,IPO Date
0,A,"Agilent Technologies, Inc.",Healthcare,Diagnostics & Research,USA,31.90B,47.37,28.57,5.04,6.11,...,0.29%,2.40,1.21M,0.73,104.83,-0.74%,893965,Aug 18/a,103.64,11/18/1999
1,AA,Alcoa Corporation,Basic Materials,Aluminum,USA,2.41B,-,-,-,0.26,...,0.23%,2.50,5.43M,1.35,13.32,4.06%,7320878,Oct 14/a,14.32,10/18/2016
2,AAAU,Perth Mint Physical Gold ETF,Financial,Exchange Traded Fund,USA,-,-,-,-,-,...,0.63%,-,642.49K,0.53,19.17,0.79%,343045,-,-,8/15/2018
3,AACG,ATA Creativity Global,Consumer Defensive,Education & Training Services,China,36.84M,-,-,-,1.61,...,2.81%,-,72.09K,0.71,1.18,-5.22%,51733,Aug 12/a,-,1/29/2008
4,AACQ,Artius Acquisition Inc.,Financial,Shell Companies,USA,706.39M,-,-,-,-,...,0.00%,-,391.66K,1.76,9.75,-0.51%,674450,-,-,9/4/2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7612,ZUO,"Zuora, Inc.",Technology,Software - Infrastructure,USA,1.23B,-,-,-,4.23,...,0.46%,2.90,1.92M,0.55,10.47,-2.79%,1070752,Sep 02/a,13.58,4/12/2018
7613,ZVO,Zovio Inc,Consumer Defensive,Education & Training Services,USA,136.41M,-,9.81,-,0.34,...,1.22%,1.00,711.36K,0.64,4.12,-5.50%,453033,Aug 03/b,-,4/15/2009
7614,ZYME,Zymeworks Inc.,Healthcare,Biotechnology,Canada,1.82B,-,-,-,59.79,...,-0.02%,1.50,420.71K,1.15,40.66,-6.03%,480639,-,24.88,4/28/2017
7615,ZYNE,"Zynerba Pharmaceuticals, Inc.",Healthcare,Drug Manufacturers - Specialty & Generic,USA,103.10M,-,-,-,-,...,0.27%,2.00,578.13K,0.48,3.57,-2.19%,283037,-,-,8/5/2015


## Arreglamos datos

### Eliminamos símbolos no numéricos

In [8]:
finviz.replace('-',np.nan, inplace=True)
finviz.replace({'%': ''}, regex=True, inplace=True)

### Arreglamos los numericos hibridos

In [9]:
def aNumerico(df, columnas):

    for columna in columnas:
        count_T, count_B, count_M, count_K = 0,0,0,0
        for idx, row in finviz.iterrows():
            try:
                trillones = (row[columna]).find('T') 
                billones = (row[columna]).find('B') 
                millones = (row[columna]).find('M')
                miles = (row[columna]).find('K')

                if  trillones > 0:
                    count_T += 1
                    row[columna] = float(row[columna][:trillones]) * 10**12
                if  billones > 0:
                    count_B += 1
                    row[columna] = float(row[columna][:billones]) * 10**9
                if  millones > 0:
                    count_M += 1
                    row[columna] = float(row[columna][:millones]) * 10**6
                if  miles > 0:
                    count_K += 1
                    row[columna] = float(row[columna][:miles]) * 10**3
            except:
                pass
            
        print(f'Columna: {columna}, T->{count_T}, B->{count_B}, M->{count_M}, K->{count_K}')

In [10]:
columnas = ['Market Cap', 'Avg Volume','Outstanding','Float',]

aNumerico(finviz, columnas)

Columna: Market Cap, T->0, B->2344, M->3039, K->0
Columna: Avg Volume, T->0, B->0, M->1702, K->5915
Columna: Outstanding, T->0, B->189, M->5197, K->0
Columna: Float, T->0, B->151, M->4675, K->0


### Pasamos a Flotante lo que sea flotante

In [11]:
def aFlotante(df, columnas):
    for columna in columnas:
        finviz[columna] = finviz[columna].astype(float)

In [12]:
columnas_no_float = ['Ticker', 'Company', 'Sector', 'Industry', 'Country', 'Market Cap',
                     'Outstanding','Float','Avg Volume','Volume', 'Earnings', 'IPO Date']

columnas_float = [col for col in list(finviz.columns) if col not in columnas_no_float]
aFlotante(finviz,columnas_float)

### Pasamos a entero lo que es entero

In [13]:
type(finviz.Volume.iloc[0])

str

In [14]:
def aEntero(df, columnas):
    for columna in columnas:
        finviz[columna] = pd.to_numeric(finviz[columna])

In [15]:
columnas_int = ['Market Cap', 'Outstanding','Float','Avg Volume','Volume']
aEntero(finviz, columnas_int)

### Pasamos a Fecha lo que se pueda pasar a fecha

In [16]:
type(finviz['IPO Date'].iloc[0])

str

In [17]:
finviz['IPO Date'] = pd.to_datetime(finviz['IPO Date'])

In [18]:
type(finviz['IPO Date'].iloc[0])

pandas._libs.tslibs.timestamps.Timestamp

### Arreglamos nombres de columnas

In [19]:
columnas_nombres = [c.replace(' ','_') for c in list(finviz.columns)]
finviz.columns = columnas_nombres

In [32]:
columnas_nombres = [c.replace('/','_') for c in list(finviz.columns)]
finviz.columns = columnas_nombres

In [33]:
finviz

,Ticker,Company,Sector,Industry,Country,Market_Cap,P_E,Fwd_P_E,PEG,P_S,...,Gap,Recom,Avg_Volume,Rel_Volume,Price,Change,Volume,Earnings,Target_Price,IPO_Date
0,A,"Agilent Technologies, Inc.",Healthcare,Diagnostics & Research,USA,3.190000e+10,47.37,28.57,5.04,6.11,...,0.29,2.4,1210000.0,0.73,104.83,-0.74,893965,Aug 18/a,103.64,1999-11-18
1,AA,Alcoa Corporation,Basic Materials,Aluminum,USA,2.410000e+09,NaN,NaN,NaN,0.26,...,0.23,2.5,5430000.0,1.35,13.32,4.06,7320878,Oct 14/a,14.32,2016-10-18
2,AAAU,Perth Mint Physical Gold ETF,Financial,Exchange Traded Fund,USA,NaN,NaN,NaN,NaN,NaN,...,0.63,NaN,642490.0,0.53,19.17,0.79,343045,NaN,NaN,2018-08-15
3,AACG,ATA Creativity Global,Consumer Defensive,Education & Training Services,China,3.684000e+07,NaN,NaN,NaN,1.61,...,2.81,NaN,72090.0,0.71,1.18,-5.22,51733,Aug 12/a,NaN,2008-01-29
4,AACQ,Artius Acquisition Inc.,Financial,Shell Companies,USA,7.063900e+08,NaN,NaN,NaN,NaN,...,0.00,NaN,391660.0,1.76,9.75,-0.51,674450,NaN,NaN,2020-09-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7612,ZUO,"Zuora, Inc.",Technology,Software - Infrastructure,USA,1.230000e+09,NaN,NaN,NaN,4.23,...,0.46,2.9,1920000.0,0.55,10.47,-2.79,1070752,Sep 02/a,13.58,2018-04-12
7613,ZVO,Zovio Inc,Consumer Defensive,Education & Training Services,USA,1.364100e+08,NaN,9.81,NaN,0.34,...,1.22,1.0,711360.0,0.64,4.12,-5.50,453033,Aug 03/b,NaN,2009-04-15
7614,ZYME,Zymeworks Inc.,Healthcare,Biotechnology,Canada,1.820000e+09,NaN,NaN,NaN,59.79,...,-0.02,1.5,420710.0,1.15,40.66,-6.03,480639,NaN,24.88,2017-04-28
7615,ZYNE,"Zynerba Pharmaceuticals, Inc.",Healthcare,Drug Manufacturers - Specialty & Generic,USA,1.031000e+08,NaN,NaN,NaN,NaN,...,0.27,2.0,578130.0,0.48,3.57,-2.19,283037,NaN,NaN,2015-08-05


## Volvemos a persistir la base arreglada

In [62]:
%%timeit
finviz.to_sql(con=sql_conn, name='finviz_ok', if_exists='replace')

2.37 s ± 168 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Pasamos los campos de texto a Varchar

In [35]:
columnas_text = ['Ticker', 'Company','Sector','Industry','Country','Earnings']

for c in  columnas_text:
    query = f'ALTER TABLE finviz_ok CHANGE {c} {c} VARCHAR(255) NULL DEFAULT NULL'
    sql_conn.execute(query)

### Agregamos un INDICE

In [36]:
query = 'ALTER TABLE finviz_ok ADD PRIMARY KEY(Ticker)'
sql_conn.execute(query)

### Eliminamos un índice

In [24]:
query = 'ALTER TABLE finviz_ok DROP PRIMARY KEY'
sql_conn.execute(query)

### Medimos performance de consulta tipo filtro

In [37]:
%%timeit
pd.read_sql('SELECT Ticker, Volume FROM finviz ORDER BY Volume ASC LIMIT 10', sql_conn)

14.6 ms ± 789 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [38]:
%%timeit
pd.read_sql('SELECT Ticker, Volume FROM finviz_ok ORDER BY Volume ASC LIMIT 10', sql_conn)

7.08 ms ± 291 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Medimos Performance de consulta agregada

In [75]:
q = 'SELECT Sector, AVG(P_E) as P_E FROM finviz_ok \
        WHERE P_E < 100 \
        GROUP BY Sector ORDER BY P_E'
pd.read_sql(q, sql_conn)

,Sector,P_E
0,Energy,14.853619
1,Financial,17.870808
2,Utilities,23.612949
3,Consumer Defensive,25.497440
4,Communication Services,26.849118
5,Basic Materials,28.194615
6,Industrials,28.543313
7,Consumer Cyclical,28.660780
8,Real Estate,31.356190
9,Healthcare,34.480000


In [82]:
q = 'SELECT Sector, Industry, AVG(P_E) as P_E, COUNT(*) AS cantidad \
        FROM finviz_ok \
        WHERE P_E < 100 \
        GROUP BY Sector, Industry ORDER BY P_E'
pd.read_sql(q, sql_conn)

,Sector,Industry,P_E,cantidad
0,Energy,Uranium,1.780000,1
1,Financial,Mortgage Finance,8.020000,11
2,Consumer Cyclical,Textile Manufacturing,8.420000,2
3,Industrials,Marine Shipping,8.507000,10
4,Energy,Oil & Gas Midstream,10.063182,44
...,...,...,...,...
141,Technology,Solar,56.084000,5
142,Industrials,Conglomerates,56.850000,3
143,Consumer Cyclical,Luxury Goods,60.670000,1
144,Financial,Shell Companies,68.675000,2


In [56]:
q = 'SELECT Sector, AVG(P_E) as P_E FROM finviz \
        WHERE P_E < 100 \
        GROUP BY Sector ORDER BY P_E'

In [57]:
%%timeit
pd.read_sql(q, sql_conn)

35.4 ms ± 638 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [54]:
q = 'SELECT Sector, AVG(P_E) as P_E FROM finviz_ok \
        WHERE P_E < 100 \
        GROUP BY Sector ORDER BY P_E'

In [55]:
%%timeit
pd.read_sql(q, sql_conn)

8.71 ms ± 188 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Comparación contra Excel

In [60]:
df = pd.read_sql('finviz_ok', sql_conn)

In [61]:
%%timeit
df.to_excel('finviz.xlsx')

9.7 s ± 1.03 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [63]:
%%timeit
pd.read_excel('finviz.xlsx')

5.55 s ± 305 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [83]:
filtro = df.loc[df.P_E < 100]
agg = pd.concat([filtro.groupby(['Sector','Industry']).P_E.mean(), filtro.groupby(['Sector','Industry']).P_E.size() ], axis=1)
agg.columns = ['P_E','Cantidad']
agg.sort_values('P_E')

,,P_E,Cantidad
Sector,Industry,,
Energy,Uranium,1.780000,1
Financial,Mortgage Finance,8.020000,11
Consumer Cyclical,Textile Manufacturing,8.420000,2
Industrials,Marine Shipping,8.507000,10
Energy,Oil & Gas Midstream,10.063182,44
...,...,...,...
Technology,Solar,56.084000,5
Industrials,Conglomerates,56.850000,3
Consumer Cyclical,Luxury Goods,60.670000,1


# Para ir trabajando

* Otros paneles
    * Precios en vivo paneles de FMP


* Historicos
    * Equity historico con batch requests ALPACA
    * Equity con yahoo
    * Buscar alternativas como investing


* Intradiarios    
    * Intradiario IEX / Alpaca


* Cadenas de Opciones
    * Ameritrade precios/puntas/Griegas
    * Yahoo